<a href="https://colab.research.google.com/github/chengulatj/googlemaps_api_reverse_geocoding/blob/main/GoogleMaps_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
locations = pd.read_excel('/content/drive/MyDrive/SL0202_Replacement_Locations.xlsx', skiprows=1)
locations.head()

,Roadway,Log Mile,IP,Location,County,Job Number,Switch,Expansion,Digital Logger,Latitude,Longitude
0,I-44,IS044W261.3C,10.6.126.170,I-44 @ Allenton/Six Flags,NaN,NaN,1,1.0,NaN,"38°30'22.99""N","90°40'40.82""W"
1,I-55,IS055N190.7Z,10.6.104.97,I-55 @ Richardson,NaN,NaN,1,1.0,NaN,"38°25'0.69""N","90°23'26.04""W"
2,I-55,IS055N198.0Z0,10.6.139.93,I-55 @ 61,NaN,NaN,1,1.0,1.0,"38°30'40.85""N","90°20'8.27""W"
3,I-55,IS055N199.1C,10.6.138.95,I-55 N of 61 (I-55 S of Reavis Barracks),NaN,NaN,1,1.0,NaN,"38°31'57.61""N","90°18'38.45""W"
4,I-55,IS055N202.7Z,10.6.100.236,I-55 @ Reavis Barracks East,NaN,NaN,1,1.0,NaN,"38°32'0.63""N","90°18'45.04""W"


In [2]:
#creating a sub-dataframe with the coordinates only
latlongs = locations[['Latitude', 'Longitude']]
latlongs.head()

,Latitude,Longitude
0,"38°30'22.99""N","90°40'40.82""W"
1,"38°25'0.69""N","90°23'26.04""W"
2,"38°30'40.85""N","90°20'8.27""W"
3,"38°31'57.61""N","90°18'38.45""W"
4,"38°32'0.63""N","90°18'45.04""W"


In [3]:
import re

# Define a helper function to convert Degree-Minute-Second (DMS) to decimal
def dms_to_dd(dms_str):
    # Convert the input to string to handle potential non-string values
    dms_str = str(dms_str)
    parts = re.split('[°\'"]+', dms_str)

    # Handle cases where the input is not in DMS format or is empty
    if len(parts) != 4:
        return float('nan')  # Return NaN for invalid inputs

    degrees = float(parts[0])
    minutes = float(parts[1])
    seconds = float(parts[2])
    direction = parts[3].strip()

    dd = degrees + minutes / 60 + seconds / 3600
    if direction in ['S', 'W']:
        dd *= -1  # South and West are negative
    return dd

# Apply the conversion to both Latitude and Longitude columns using .loc to avoid warnings
latlongs.loc[:, 'Latitude_dec'] = latlongs['Latitude'].apply(dms_to_dd)
latlongs.loc[:, 'Longitude_dec'] = latlongs['Longitude'].apply(dms_to_dd)

<ipython-input-3-3726e582e0ad>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlongs.loc[:, 'Latitude_dec'] = latlongs['Latitude'].apply(dms_to_dd)
<ipython-input-3-3726e582e0ad>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlongs.loc[:, 'Longitude_dec'] = latlongs['Longitude'].apply(dms_to_dd)


In [4]:
# Display the sample result
latlongs[['Latitude', 'Longitude', 'Latitude_dec', 'Longitude_dec']].head()

,Latitude,Longitude,Latitude_dec,Longitude_dec
0,"38°30'22.99""N","90°40'40.82""W",38.506386,-90.678006
1,"38°25'0.69""N","90°23'26.04""W",38.416858,-90.390567
2,"38°30'40.85""N","90°20'8.27""W",38.511347,-90.335631
3,"38°31'57.61""N","90°18'38.45""W",38.532669,-90.310681
4,"38°32'0.63""N","90°18'45.04""W",38.533508,-90.312511


In [5]:
!pip install -U googlemaps

In [6]:
import googlemaps
import pandas as pd
import time

# Read the Generated API key from the file stored in Google Drive
with open('/content/drive/MyDrive/KEY.txt') as f:
    for line in f:
        key, value = line.strip().split('=')
        if key == 'API_KEY':
            API_KEY = value

# Initialize Google Maps client with the API key read from the file
gmaps = googlemaps.Client(key=API_KEY)

In [7]:
# Function to reverse geocode using Google Maps API and extract the county
def get_county_google(lat, lon):
    try:
        # Check if latitude and longitude are valid
        if pd.isna(lat) or pd.isna(lon):
            return 'Invalid Coordinates'
        if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
            return 'Invalid Lat/Long Range'

        # Print coordinates for debugging
        print(f"Processing Lat: {lat}, Lon: {lon}")

        # Use reverse geocoding to get the location data
        result = gmaps.reverse_geocode((lat, lon))

        # Loop through the address components to find the county
        for component in result[0]['address_components']:
            if 'administrative_area_level_2' in component['types']:
                return component['long_name']  # Return the county name

        return 'County not found'
    except Exception as e:
        print(f"Error: {e}")
        return 'Error'


In [8]:
# Apply reverse geocoding to get the county for each lat-long pair
latlongs.loc[:,'County'] = latlongs.apply(lambda row: get_county_google(row['Latitude_dec'], row['Longitude_dec']), axis=1)

Processing Lat: 38.50638611111111, Lon: -90.67800555555556
Processing Lat: 38.41685833333333, Lon: -90.39056666666667
Processing Lat: 38.51134722222222, Lon: -90.33563055555555
Processing Lat: 38.532669444444444, Lon: -90.31068055555555
Processing Lat: 38.53350833333333, Lon: -90.3125111111111
Processing Lat: 38.6083, Lon: -90.21219722222223
Processing Lat: 38.441066666666664, Lon: -90.38112222222222
Processing Lat: 38.44199444444444, Lon: -90.381475
Processing Lat: 38.48585, Lon: -90.35621666666665
Processing Lat: 38.57448055555556, Lon: -90.23363333333333
Processing Lat: 38.61277777777778, Lon: -90.19723888888889
Processing Lat: 38.65386111111111, Lon: -90.55827222222221
Processing Lat: 38.638983333333336, Lon: -90.49841666666667
Processing Lat: 38.62918888888889, Lon: -90.30698611111112
Processing Lat: 38.631838888888886, Lon: -90.28544166666667
Processing Lat: 38.630825, Lon: -90.22758333333333
Processing Lat: 38.715138888888895, Lon: -90.70363055555556
Processing Lat: 38.710186111

<ipython-input-8-76036eda2146>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latlongs.loc[:,'County'] = latlongs.apply(lambda row: get_county_google(row['Latitude_dec'], row['Longitude_dec']), axis=1)


In [9]:
# Display the result
latlongs[['Latitude', 'Longitude', 'Latitude_dec', 'Longitude_dec', 'County']].head()

,Latitude,Longitude,Latitude_dec,Longitude_dec,County
0,"38°30'22.99""N","90°40'40.82""W",38.506386,-90.678006,St. Louis County
1,"38°25'0.69""N","90°23'26.04""W",38.416858,-90.390567,Jefferson County
2,"38°30'40.85""N","90°20'8.27""W",38.511347,-90.335631,St. Louis County
3,"38°31'57.61""N","90°18'38.45""W",38.532669,-90.310681,St. Louis County
4,"38°32'0.63""N","90°18'45.04""W",38.533508,-90.312511,St. Louis County


In [10]:
#adding the newly generated column into the original dataset
locations['County'] = latlongs['County']
locations.head()

,Roadway,Log Mile,IP,Location,County,Job Number,Switch,Expansion,Digital Logger,Latitude,Longitude
0,I-44,IS044W261.3C,10.6.126.170,I-44 @ Allenton/Six Flags,St. Louis County,NaN,1,1.0,NaN,"38°30'22.99""N","90°40'40.82""W"
1,I-55,IS055N190.7Z,10.6.104.97,I-55 @ Richardson,Jefferson County,NaN,1,1.0,NaN,"38°25'0.69""N","90°23'26.04""W"
2,I-55,IS055N198.0Z0,10.6.139.93,I-55 @ 61,St. Louis County,NaN,1,1.0,1.0,"38°30'40.85""N","90°20'8.27""W"
3,I-55,IS055N199.1C,10.6.138.95,I-55 N of 61 (I-55 S of Reavis Barracks),St. Louis County,NaN,1,1.0,NaN,"38°31'57.61""N","90°18'38.45""W"
4,I-55,IS055N202.7Z,10.6.100.236,I-55 @ Reavis Barracks East,St. Louis County,NaN,1,1.0,NaN,"38°32'0.63""N","90°18'45.04""W"


In [11]:
#saving the updated dataframe to a file
locations.to_excel('/content/drive/MyDrive/SL0202_Replacement_Locations_with_County2.xlsx', index=False)